In [0]:
import os
cwd = os.getcwd()
cwd

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
'''#!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil -q
!pip install psutil -q
!pip install humanize -q
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()'''


import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
#Create Keras save folder

#os.chdir(r'C:\Users\Murat Eliby\Desktop\wd')
if not os.path.exists('/content/keras'):
    os.makedirs('/content/keras')

In [0]:
#Empty keras folder

import os, shutil
folder = '/content/keras'
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [0]:
# Blood Donations Prediction 23.01.2019

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
!curl https://raw.githubusercontent.com/muke888/Portfolio_DrivenData.org/master/001.Blood%20Donations%20Prediction/BloodDonations_TrainData.csv -o TrainData.csv
!curl https://raw.githubusercontent.com/muke888/Portfolio_DrivenData.org/master/001.Blood%20Donations%20Prediction/BloodDonations_TestData.csv  -o TestData.csv 
datatrain = pd.read_csv('TrainData.csv')
datatest = pd.read_csv('TestData.csv')
datatrain.info()

In [0]:
datatrain.head()

In [0]:
datatrain_x = datatrain.drop(['Made Donation in March 2007','Total Volume Donated (c.c.)'],	 axis=1)
datatest = datatest.drop('Total Volume Donated (c.c.)', axis=1)

In [0]:
combined_data = pd.concat([datatrain_x, datatest], ignore_index=True)
combined_data.info()

In [0]:
scaler_X = combined_data.iloc[:,1:].values
scaler_X[0]

In [0]:
# Feature Scaling - StandardScaler

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(scaler_X)


X = datatrain_x.iloc[:,1:].values
y = datatrain.iloc[:, 5].values

In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)#, stratify=y)

#scale_transform based on all data
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)


In [0]:
x = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)
print(x.shape,y.shape)

In [0]:
!pip install -q talos

In [0]:
#input dimension
X_train.shape[1]

In [0]:
import tensorflow as tf
import talos as ta

from talos.model import lr_normalizer
from talos.model.early_stopper import early_stopper

LeakyReLU = tf.keras.layers.LeakyReLU(alpha=0.3)
ELU = tf.keras.layers.ELU(alpha=1.0)
ReLU = tf.keras.layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0)

para = {
    'dense1_neuron': [2, 3],
    'dense2_neuron': [2, 3],
    'dense3_neuron': [2, 3],
    'activation':[LeakyReLU, ELU],
    'batch_size': [27, 16, 8],
    'weight_regulizer':[None],
    'dropout': [0, 0.1],
    'epochs': [500],
    #'optimizer': [Adam], #Nadam
    'kernel_initializer': ['glorot_uniform'],
    'last_activation': ['sigmoid']
}


def biclass_fn_gpu1(X_train, y_train, X_test, y_test, params):
  
    tf.keras.backend.clear_session()
    #dropout = float(params['dropout'])
    #dense1_neuron = int(params['dense1_neuron'])
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=params['dense1_neuron'],input_dim=3,kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(params['activation'])
    model.add(tf.keras.layers.Dropout(params['dropout']))

    model.add(tf.keras.layers.Dense(units=params['dense2_neuron'],kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(params['activation'])
    model.add(tf.keras.layers.Dropout(params['dropout']))    
    
    model.add(tf.keras.layers.Dense(units=params['dense3_neuron'],kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(params['activation'])
    model.add(tf.keras.layers.Dropout(params['dropout'])) 
    
    model.add(tf.keras.layers.Dense(units=1,kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("sigmoid"))
    
    
    model.compile(
        optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy'])


    out = model.fit(
        X_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'],
        verbose=0,
        validation_data=[X_test, y_test] #,callbacks=[early_stopper(params['epochs'], mode='moderate')]
    )
    
    return out, model

In [0]:
import tensorflow as tf
import talos as ta
import os

def biclass_fn_tpu(X_train, y_train, X_test, y_test, params):
  
    tf.keras.backend.clear_session()
    #dropout = float(params['dropout'])
    #dense1_neuron = int(params['dense1_neuron'])
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=params['dense1_neuron'],input_dim=3,kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(params['activation']))
    model.add(tf.keras.layers.Dropout(params['dropout']))

    model.add(tf.keras.layers.Dense(units=params['dense2_neuron'],kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(params['activation']))
    model.add(tf.keras.layers.Dropout(params['dropout']))    
    
    model.add(tf.keras.layers.Dense(units=params['dense3_neuron'],kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(params['activation']))
    model.add(tf.keras.layers.Dropout(params['dropout'])) 
    
    model.add(tf.keras.layers.Dense(units=1,kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("sigmoid"))
    
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(
        model,
        strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(
            tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
        )
    )
    tpu_model.compile(
        optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy']
    )


    out = tpu_model.fit(
        X_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'],
        verbose=0,
        validation_data=[X_test, y_test]
    )
    
    return out, tpu_model.sync_to_cpu()

In [0]:
scan_results = ta.Scan(x, y, para, biclass_fn_tpu) #grid_downsample=0.5

In [0]:
import tensorflow as tf
import talos as ta
import os
#from talos.model.early_stopper import early_stopper

para = {
    'dense1_neuron': [2, 3],
    'dense2_neuron': [2, 3],
    'dense3_neuron': [2, 3],
    'activation':['elu', 'selu'], #'tanh' ,'leaky_relu','crelu','selu,','relu6','relu'
    'last activation':['sigmoid'], #'softmax'
    'dropout': [0],
    'batch_size': [32,24],
    'weight_regulizer':[None],
    'epochs': [450,475,500,525,550,575,600],
    'optimizer': ['Adam'],
    'kernel_initializer': ['glorot_uniform']
}

#'hidden_layers':[0, 1, 2, 3]
#'first_neuron':[4, 8, 16, 32, 64, 128],


def biclass_fn_gpu(X_train, y_train, X_test, y_test, params):
  
    tf.keras.backend.clear_session()
    #dropout = float(params['dropout'])
    #dense1_neuron = int(params['dense1_neuron'])
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=params['dense1_neuron'],input_dim=3,kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(params['activation']))
    model.add(tf.keras.layers.Dropout(params['dropout']))

    model.add(tf.keras.layers.Dense(units=params['dense2_neuron'],kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(params['activation']))
    model.add(tf.keras.layers.Dropout(params['dropout']))    
    
    model.add(tf.keras.layers.Dense(units=params['dense3_neuron'],kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(params['activation']))
    model.add(tf.keras.layers.Dropout(params['dropout'])) 
    
    model.add(tf.keras.layers.Dense(units=1,kernel_initializer="glorot_uniform"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(params['last activation']))
    
    
    model.compile(
        optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
        loss=tf.keras.losses.binary_crossentropy,
        metrics=['accuracy']) # accuracy


    out = model.fit(
        X_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'],
        verbose=0,
        validation_data=[X_test, y_test]
        #,callbacks=early_stopper(params['epochs'], mode='strict')
    ) 
    
    return out, model

In [0]:
#tf.keras.backend.clear_session()
scan_results = ta.Scan(x, y, para, biclass_fn_gpu) #grid_downsample=0.5

hkl.dump(scan_results, '/content/drive/My Drive/Colab Notebooks/DrivenData.org/Models_Weights/scan_results_8.hkl', mode='w')

In [0]:
scan_results.data.head(10)

In [0]:
scan_results.data.to_csv('results.csv')

In [0]:
# highest val_acc model_id  & index

model_id_index = scan_results.data['val_acc'].astype('float').argmax() - 1
model_id = model_id_index + 1
model_id

In [0]:
scan_results.details

In [0]:
scan_results.peak_epochs_df.head()

In [0]:
scan_results.peak_epochs_df.to_csv('peak_epochs_df.csv')

In [0]:
!pip install -q hickle
import hickle as hkl
import tensorflow as tf

In [0]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

In [0]:
hkl.dump(scan_results, 'scan_results_80split.hkl', mode='w')

In [0]:
#Save models and weights 

hkl.dump(scan_results.saved_models, 'models4.hkl', mode='w')
#hkl.dump(scan_results.saved_models, 'models_zip.hkl', mode='w', compression='gzip')
hkl.dump(scan_results.saved_weights, 'weights4.hkl', mode='w')
#hkl.dump(scan_results.saved_models, 'weights_zip.hkl', mode='w', compression='gzip')

In [0]:
import os
# Compare filesizes
print('Models: %i bytes' % os.path.getsize('models3.hkl'))
print('Weights:   %i bytes' % os.path.getsize('weights3.hkl'))

In [0]:
scan_results = hkl.load('/content/drive/My Drive/Colab Notebooks/DrivenData.org/Models_Weights/scan_results_8.hkl') #5nonstratified


In [0]:
#scan_results.x

# Load data from GDrive

loaded_models = hkl.load('/content/drive/My Drive/Colab Notebooks/DrivenData.org/Models_Weights/models4.hkl')
loaded_weights = hkl.load('/content/drive/My Drive/Colab Notebooks/DrivenData.org/Models_Weights/weights4.hkl')

# Check the two are the same file
#assert scan_results.saved_models == loaded_models

In [0]:
tf.keras.backend.clear_session()
from tensorflow.keras.models import model_from_json

#model = model_from_json(scan_results.saved_models[126])
model = model_from_json(loaded_models[147])

#model.set_weights(scan_results.saved_weights[126])
model.set_weights(loaded_weights[147])
model.summary()

In [0]:
#model.save('./best_model.h5')
#from google.colab import files
#files.download('./best_model.h5')

!pip install -q keras_sequential_ascii
from keras_sequential_ascii import keras2ascii
keras2ascii(model)

In [0]:
test_features = datatest.iloc[:,1:].values
test_features = sc.transform(test_features)
test_features[0]

In [0]:
test_predict = model.predict_proba(test_features)
df=pd.DataFrame(data=test_predict)
df.columns = ['Made Donation in March 2007']
df.index = [659,	276,	263,	303,	83,	500,	530,	244,	249,	728,	129,	534,	317,	401,	696,	192,	176,	571,	139,	423,	563,	56,	528,	101,	467,	382,	466,	294,	512,	659,	389,	487,	701,	419,	536,	240,	508,	515,	283,	650,	65,	228,	741,	297,	464,	63,	231,	28,	248,	357,	300,	726,	680,	520,	254,	582,	143,	98,	1,	221,	352,	64,	138,	745,	64,	688,	623,	289,	174,	690,	105,	427,	48,	14,	657,	301,	455,	579,	722,	98,	491,	303,	466,	65,	300,	9,	622,	323,	289,	568,	290,	156,	464,	426,	306,	4,	12,	187,	406,	96,	509,	733,	548,	478,	501,	127,	199,	299,	162,	235,	23,	473,	487,	683,	303,	309,	569,	34,	686,	84,	733,	537,	181,	453,	67,	161,	307,	703,	181,	246,	316,	278,	346,	545,	419,	694,	622,	663,	262,	461,	373,	233,	466,	207,	263,	16,	513,	449,	429,	701,	632,	529,	245,	344,	353,	241,	633,	624,	726,	189,	138,	402,	511,	590,	334,	447,	119,	389,	644,	423,	131,	405,	82,	643,	156,	617,	574,	272,	613,	545,	685,	570,	537,	691,	85,	483,	455,	93,	744,	33,	321,	523,	426,	196,	301,	103,	224,	454,	585,	154]
df.to_csv('Talos3.csv')

In [0]:
#test_predict = model.predict(test_features)
#df=pd.DataFrame(data=test_predict)
#df.to_csv('Talos3.csv')

In [0]:
corr = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DrivenData.org/correlations.csv')

In [0]:
corr.drop("Score", axis=1).apply(lambda x: x.corr(corr.Score))